In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.functional import F

import wandb

print(torch.__version__)
print(torch.backends.mps.is_available())

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")


1.12.1
True
Using mps device


In [7]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [19]:
X, y = df.drop('Outcome', axis=1).values, df.Outcome.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [34]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=10, hidden3= 5, out_features=2):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.f_connected3 = nn.Linear(hidden2, hidden3)
        self.out = nn.Linear(hidden3, out_features)

    def forward(self,x):
        x = F.relu(self.f_connected1(x))
        x = F.relu(self.f_connected2(x))
        x = F.relu(self.f_connected3(x))
        x = self.out(x)
        
        return x

model = NeuralNetwork()
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

NeuralNetwork(
  (f_connected1): Linear(in_features=8, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=10, bias=True)
  (f_connected3): Linear(in_features=10, out_features=5, bias=True)
  (out): Linear(in_features=5, out_features=2, bias=True)
)


In [35]:
epochs = 500
final_losses = []

for i in range(epochs):
    i += 1
    y_pred = model.forward(X_train)
    loss = loss_fn(y_pred, y_train)
    final_losses.append(loss)

    if i % 10 == 0:
        print(f'Epoch {i}: {loss.item()}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch 10: 0.5775846838951111
Epoch 20: 0.49366334080696106
Epoch 30: 0.4600379168987274
Epoch 40: 0.44300633668899536
Epoch 50: 0.4259662330150604
Epoch 60: 0.41066673398017883
Epoch 70: 0.39965084195137024
Epoch 80: 0.3958785831928253
Epoch 90: 0.3857158124446869
Epoch 100: 0.3738996684551239
Epoch 110: 0.3661211133003235
Epoch 120: 0.3612293303012848
Epoch 130: 0.3559854328632355
Epoch 140: 0.356245219707489
Epoch 150: 0.35114189982414246
Epoch 160: 0.3439518213272095
Epoch 170: 0.33825650811195374
Epoch 180: 0.3412216901779175
Epoch 190: 0.33816710114479065
Epoch 200: 0.3372369408607483
Epoch 210: 0.32699310779571533
Epoch 220: 0.32468509674072266
Epoch 230: 0.32271113991737366
Epoch 240: 0.32409554719924927
Epoch 250: 0.3335336744785309
Epoch 260: 0.323016494512558
Epoch 270: 0.31942862272262573
Epoch 280: 0.3175789415836334
Epoch 290: 0.3228004574775696
Epoch 300: 0.34253451228141785
Epoch 310: 0.3170837163925171
Epoch 320: 0.314464271068573
Epoch 330: 0.3124334514141083
Epoch 340

In [36]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_scatter(x=list(range(epochs)), y=list(map(lambda x: x.item(), final_losses)))
fig.update_layout(xaxis_title='Epoch', yaxis_title='Loss')

In [37]:
predictions=[]

with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())

from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)
score

0.8246753246753247

In [27]:
# From pytorch tutorial - requires dataloader

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 5 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(0) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 0.623051  [   16/  537]
loss: 0.710709  [   96/  537]
loss: 0.689016  [  176/  537]
loss: 0.609918  [  256/  537]
loss: 0.759521  [  336/  537]
loss: 0.595187  [  416/  537]
loss: 0.662290  [  496/  537]
Test Error: 
 Accuracy: 64.5%, Avg loss: 0.652138 

Epoch 2
-------------------------------
loss: 0.587295  [   16/  537]
loss: 0.740669  [   96/  537]
loss: 0.741760  [  176/  537]
loss: 0.663411  [  256/  537]
loss: 0.784175  [  336/  537]
loss: 0.586118  [  416/  537]
loss: 0.782245  [  496/  537]
Test Error: 
 Accuracy: 64.5%, Avg loss: 0.655491 

Epoch 3
-------------------------------
loss: 0.701184  [   16/  537]
loss: 0.700722  [   96/  537]
loss: 0.623587  [  176/  537]
loss: 0.583861  [  256/  537]
loss: 0.583432  [  336/  537]
loss: 0.542940  [  416/  537]
loss: 0.744505  [  496/  537]
Test Error: 
 Accuracy: 64.5%, Avg loss: 0.655557 

Epoch 4
-------------------------------
loss: 0.701636  [   16/  537]
loss: 0.662767  [   96/ 

In [15]:
classes = [
    'Healthy', 
    'Diabetic'
]

model.eval()
x, y = torch.tensor(X_test[0]).to(device), torch.tensor(y_test[0]).to(device)
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[int(pred[0].argmax(0))], classes[int(y)]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Healthy", Actual: "Healthy"


/var/folders/49/ymyjgyx173l0zz0x9x2h95jh0000gn/T/ipykernel_72350/2717208966.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x, y = torch.tensor(X_test[0]).to(device), torch.tensor(y_test[0]).to(device)
